In [10]:
import torch 
import torch.nn as nn
import torch.optim as optim

latent_dim = 100
hidden_dim = 32
data_dim = 1
batch_size = 16
lr = 2e-4

In [11]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            
            nn.Linear(latent_dim, hidden_dim),
            nn.ReLU(inplace=True),
            
            nn.Linear(hidden_dim, data_dim)
        )
    def forward(self,z):
        return self.model(z)
       
        

In [12]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
           
            nn.Linear(data_dim, hidden_dim),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()
        )
    def forward(self,x):
        return self.model(x);

In [13]:
G = Generator()
D = Discriminator()

criterion = nn.BCELoss()

opt_G = optim.Adam(G.parameters(), lr=lr)
opt_D = optim.Adam(D.parameters(), lr=lr)

In [14]:
real_data = torch.randn(batch_size,data_dim)
real_lables = torch.ones(batch_size,1)

opt_D.zero_grad()

out_real = D(real_data)
loss_real = criterion(out_real,real_lables)

z = torch.randn(batch_size,latent_dim)
with torch.no_grad():
    fake_data = G(z)
out_fake = D(fake_data)
fake_lables = torch.zeros(batch_size,1)
loss_fake = criterion(out_fake,fake_lables)

loss_D = loss_real+loss_fake
loss_D.backward()
opt_D.step()

In [15]:
opt_G.zero_grad()
z = torch.randn(batch_size,latent_dim)
fake = G(z)
out_Df = D(fake)
loss_G = criterion(out_Df,real_lables) 
loss_G.backward()
opt_G.step()

In [16]:
print(f"D loss: {loss_D.item():.3f}, G loss: {loss_G.item():.3f}")
print(f"real_data shape: {real_data.shape}, fake_data shape: {fake.shape}")

D loss: 1.420, G loss: 0.634
real_data shape: torch.Size([16, 1]), fake_data shape: torch.Size([16, 1])
